In [ ]:
import pandas as pd
from linearmodels.panel import PanelOLS
import statsmodels.api as sm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
DATA_DIR = '../data/'
os.mkdir(DATA_DIR, exist_ok=True)

* Still missing some crime files, will fix later today, i hate markdown styling sometimes
### required datafiles:
/data/vote_2010.csv     [Download](https://www.verkiezingsuitslagen.nl/uitslag/verkiezing/TK20100609/gemeenten)  
/data/vote_2012.csv     [Download](https://www.verkiezingsuitslagen.nl/uitslag/verkiezing/TK20120912/gemeenten)  
/data/vote_2017.csv     [Download](https://www.verkiezingsuitslagen.nl/uitslag/verkiezing/TK20170315/gemeenten)  
/data/vote_2021.csv     [Download](https://www.verkiezingsuitslagen.nl/uitslag/verkiezing/TK20210317/gemeenten)  
/data/vote_2023.csv     [Download](https://www.verkiezingsuitslagen.nl/uitslag/verkiezing/TK20231122/gemeenten)  
/data/vote_2025.csv     [Download](https://www.verkiezingsuitslagen.nl/uitslag/verkiezing/TK20251029/gemeenten)  
/data/cleaned_crime/crime_2010_2024.csv


In [ ]:
# WIP, I will update it to also rename the files later.


# generate cleaned_crime/crime_2010_2024.csv

# Merge the crime files from 2010 to 2024, as one can only download 1000000 cells from cbs
# files = ['10111213.csv', '14151617.csv', '18192021.csv', '222324.csv']
# dataframes = [pd.read_csv(file) for file in files]
# merged_df = pd.concat(dataframes, ignore_index=True)
# merged_df.drop_duplicates(inplace=True)
# merged_df.to_csv('crime_2010_2024.csv', index=False)

In [20]:
# Mapping of filenames to years

file_years = {
    "vote_2010.csv": 2010,
    "vote_2012.csv": 2012,
    "vote_2017.csv": 2017,
    "vote_2021.csv": 2021,
    "vote_2023.csv": 2023,
    "vote_2025.csv": 2025,
}

# List of cleaned dataframes
dfs = []

# Load and clean each file
for filename, year in file_years.items():

    # Read CSV (semicolon-separated)
    df = pd.read_csv(DATA_DIR + filename, sep=";", engine="python")

    # Add election year column
    df["Jaar"] = year

    # Clean column names for consistency
    df.columns = df.columns.str.strip()

    # Clean municipality names (remove leading/trailing quotes)
    df["Gemeente"] = df["Gemeente"].str.replace("'", "", regex=False)

    # Convert vote counts to numeric
    df["AantalStemmen"] = pd.to_numeric(df["AantalStemmen"], errors="coerce")

    # Convert seat counts to numeric
    df["AantalZetels"] = pd.to_numeric(df["AantalZetels"], errors="coerce")

    # Append to list
    dfs.append(df)

# combine all years into a single DataFrame
votes_panel_raw = pd.concat(dfs, ignore_index=True)

print(votes_panel_raw.head())
print(votes_panel_raw.tail())

       Gemeente   Code  AmsterdamseCode                           Partij  \
0  s-Gravenhage  G0518          11434.0  Partij van de Arbeid (P.v.d.A.)   
1  s-Gravenhage  G0518          11434.0                              VVD   
2  s-Gravenhage  G0518          11434.0    PVV (Partij voor de Vrijheid)   
3  s-Gravenhage  G0518          11434.0              Democraten 66 (D66)   
4  s-Gravenhage  G0518          11434.0                       GROENLINKS   

   AantalStemmen  AantalZetels  Unnamed: 6  Jaar  
0        63042.0          30.0         NaN  2010  
1        59925.0          31.0         NaN  2010  
2        43725.0          24.0         NaN  2010  
3        28075.0          10.0         NaN  2010  
4        22687.0          10.0         NaN  2010  
      Gemeente   Code  AmsterdamseCode                  Partij  AantalStemmen  \
52739   Zwolle  G0193          10093.0           Piratenpartij           98.0   
52740   Zwolle  G0193          10093.0  LP (Libertaire Partij)           64

In [21]:
# --------------------------------------------------------------
# Compute total votes per municipality-year
# --------------------------------------------------------------

total_votes = (
    votes_panel_raw.groupby(["Code", "Gemeente", "Jaar"], as_index=False)["AantalStemmen"]
                   .sum()
                   .rename(columns={"AantalStemmen": "TotalVotes"})
)

# --------------------------------------------------------------
# Compute vote share for each party
# --------------------------------------------------------------

votes_panel = pd.merge(
    votes_panel_raw,
    total_votes,
    on=["Code", "Gemeente", "Jaar"],
    how="left"
)

votes_panel["VoteShare"] = votes_panel["AantalStemmen"] / votes_panel["TotalVotes"]

print(votes_panel.head())


       Gemeente   Code  AmsterdamseCode                           Partij  \
0  s-Gravenhage  G0518          11434.0  Partij van de Arbeid (P.v.d.A.)   
1  s-Gravenhage  G0518          11434.0                              VVD   
2  s-Gravenhage  G0518          11434.0    PVV (Partij voor de Vrijheid)   
3  s-Gravenhage  G0518          11434.0              Democraten 66 (D66)   
4  s-Gravenhage  G0518          11434.0                       GROENLINKS   

   AantalStemmen  AantalZetels  Unnamed: 6  Jaar  TotalVotes  VoteShare  
0        63042.0          30.0         NaN  2010    267584.0   0.235597  
1        59925.0          31.0         NaN  2010    267584.0   0.223948  
2        43725.0          24.0         NaN  2010    267584.0   0.163407  
3        28075.0          10.0         NaN  2010    267584.0   0.104920  
4        22687.0          10.0         NaN  2010    267584.0   0.084785  


In [22]:
# output the cleaned and merged votes panel
output_folder = DATA_DIR + "cleaned_votes"

# If the folder does not exist, create it
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Created folder: {output_folder}")
else:
    print(f"Folder already exists: {output_folder}")

# --------------------------------------------------------------
# 2. Save the merged + cleaned votes panel into that folder
# --------------------------------------------------------------

output_path = os.path.join(output_folder, "votes_panel_cleaned.csv")
votes_panel.to_csv(output_path, index=False, sep=";")

print(f"Cleaned voting data exported to: {output_path}")

Folder already exists: ../data/cleaned_votes
Cleaned voting data exported to: ../data/cleaned_votes/votes_panel_cleaned.csv


In [23]:

file_path = DATA_DIR + "cleaned_crime/crime_2010_2024.csv"

rows = []
with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue  # skip empty lines
        
        # Remove leading and trailing double quotes for the whole line
        if line.startswith('"') and line.endswith('"'):
            line = line[1:-1]
        
        # Split by semicolon
        parts = line.split(";")
        
        # Clean each field: remove internal quotes and surrounding spaces
        parts = [p.replace('"', "").strip() for p in parts]
        
        rows.append(parts)

# First row = header, the rest = data
header = rows[0]
data_rows = rows[1:]

crime_raw = pd.DataFrame(data_rows, columns=header)

crime_raw.head()


,ID,SoortMisdrijf,RegioS,Perioden,TotaalGeregistreerdeMisdrijven_1,GeregistreerdeMisdrijvenRelatief_2,GeregistreerdeMisdrijvenPer1000Inw_3,TotaalOpgehelderdeMisdrijven_4,OpgehelderdeMisdrijvenRelatief_5,RegistratiesVanVerdachten_6
0,285,T001161,GM1680,2010JJ00,805,100,31.5,180,22.6,215
1,286,T001161,GM1680,2011JJ00,945,100,36.7,240,25.3,310
2,287,T001161,GM1680,2012JJ00,940,100,36.6,200,21.1,235
3,288,T001161,GM1680,2013JJ00,785,100,30.8,190,24.1,285
4,300,T001161,GM0738,2010JJ00,470,100,37.1,90,19.2,105


In [24]:

# --------------------------------------------------------------
# 1. Extract year from 'Perioden' (e.g. "2010JJ00" -> 2010)
# --------------------------------------------------------------
crime_raw["Jaar"] = crime_raw["Perioden"].str[:4].astype(int)

# --------------------------------------------------------------
# 2. Convert municipality code from "GMxxxx" to "Gxxxx"
#    so it matches the 'Code' in your votes_panel
# --------------------------------------------------------------
crime_raw["Code"] = crime_raw["RegioS"].str.replace("GM", "G", regex=False)

# --------------------------------------------------------------
# 3. Convert crime columns to numeric
# --------------------------------------------------------------
crime_raw["Crime_total"] = pd.to_numeric(
    crime_raw["TotaalGeregistreerdeMisdrijven_1"],
    errors="coerce"
)

crime_raw["Crime_rate"] = pd.to_numeric(
    crime_raw["GeregistreerdeMisdrijvenPer1000Inw_3"],
    errors="coerce"
)

# --------------------------------------------------------------
# 4. Keep only the columns we need for panel regression
# --------------------------------------------------------------
crime_panel = crime_raw[["Code", "Jaar", "Crime_total", "Crime_rate"]]

crime_panel.head()


,Code,Jaar,Crime_total,Crime_rate
0,G1680,2010,805.0,31.5
1,G1680,2011,945.0,36.7
2,G1680,2012,940.0,36.6
3,G1680,2013,785.0,30.8
4,G0738,2010,470.0,37.1


In [25]:
# --------------------------------------------------------------
# Merge municipal voting data with municipal crime data
# --------------------------------------------------------------
panel = pd.merge(
    votes_panel,
    crime_panel,
    on=["Code", "Jaar"],
    how="inner"    # Only keep rows that exist in both datasets
)

panel.head()


,Gemeente,Code,AmsterdamseCode,Partij,AantalStemmen,AantalZetels,Unnamed: 6,Jaar,TotalVotes,VoteShare,Crime_total,Crime_rate
0,s-Gravenhage,G0518,11434.0,Partij van de Arbeid (P.v.d.A.),63042.0,30.0,NaN,2010,267584.0,0.235597,58085.0,118.9
1,s-Gravenhage,G0518,11434.0,Partij van de Arbeid (P.v.d.A.),63042.0,30.0,NaN,2010,267584.0,0.235597,33595.0,68.8
2,s-Gravenhage,G0518,11434.0,Partij van de Arbeid (P.v.d.A.),63042.0,30.0,NaN,2010,267584.0,0.235597,31385.0,64.2
3,s-Gravenhage,G0518,11434.0,Partij van de Arbeid (P.v.d.A.),63042.0,30.0,NaN,2010,267584.0,0.235597,1265.0,2.6
4,s-Gravenhage,G0518,11434.0,Partij van de Arbeid (P.v.d.A.),63042.0,30.0,NaN,2010,267584.0,0.235597,30120.0,61.7


In [26]:
# check lengths of datasets
print(len(votes_panel), len(crime_panel), len(panel))

52744 522000 3342675


In [27]:
# check for duplicates in crime_panel
crime_panel.groupby(["Code", "Jaar"]).size().value_counts()

75    6960
Name: count, dtype: int64

In [28]:
# only keep total crime records "T001161"
crime_total = crime_raw[ crime_raw["SoortMisdrijf"] == "T001161" ].copy()
crime_total.groupby(["RegioS", "Perioden"]).size().value_counts()


1    6960
Name: count, dtype: int64

In [29]:
# clean up Jaar and Code columns
crime_total["Jaar"] = crime_total["Perioden"].str[:4].astype(int)
crime_total["Code"] = crime_total["RegioS"].str.replace("GM", "G", regex=False)

# convert crime columns to numeric
crime_total["Crime_total"] = pd.to_numeric(
    crime_total["TotaalGeregistreerdeMisdrijven_1"], errors="coerce"
)

crime_total["Crime_rate"] = pd.to_numeric(
    crime_total["GeregistreerdeMisdrijvenPer1000Inw_3"], errors="coerce"
)

crime_panel = crime_total[["Code", "Jaar", "Crime_total", "Crime_rate"]]

In [30]:
crime_panel.groupby(["Code", "Jaar"]).size().value_counts()

crime_panel.head()


,Code,Jaar,Crime_total,Crime_rate
0,G1680,2010,805.0,31.5
1,G1680,2011,945.0,36.7
2,G1680,2012,940.0,36.6
3,G1680,2013,785.0,30.8
4,G0738,2010,470.0,37.1


In [31]:
# --------------------------------------------------------------
# Merge municipal voting data with cleaned municipal crime data
# --------------------------------------------------------------
panel = pd.merge(
    votes_panel,
    crime_panel,
    on=["Code", "Jaar"],
    how="inner"   # keep only municipality-years present in both
)

print("Rows in votes_panel:", len(votes_panel))
print("Rows in crime_panel:", len(crime_panel))
print("Rows in merged panel:", len(panel))

panel.head()


Rows in votes_panel: 52744
Rows in crime_panel: 6960
Rows in merged panel: 44569


,Gemeente,Code,AmsterdamseCode,Partij,AantalStemmen,AantalZetels,Unnamed: 6,Jaar,TotalVotes,VoteShare,Crime_total,Crime_rate
0,s-Gravenhage,G0518,11434.0,Partij van de Arbeid (P.v.d.A.),63042.0,30.0,NaN,2010,267584.0,0.235597,58085.0,118.9
1,s-Gravenhage,G0518,11434.0,VVD,59925.0,31.0,NaN,2010,267584.0,0.223948,58085.0,118.9
2,s-Gravenhage,G0518,11434.0,PVV (Partij voor de Vrijheid),43725.0,24.0,NaN,2010,267584.0,0.163407,58085.0,118.9
3,s-Gravenhage,G0518,11434.0,Democraten 66 (D66),28075.0,10.0,NaN,2010,267584.0,0.104920,58085.0,118.9
4,s-Gravenhage,G0518,11434.0,GROENLINKS,22687.0,10.0,NaN,2010,267584.0,0.084785,58085.0,118.9


# Subquestion 1
“To what extent do changes in regional crime rates predict shifts in electoral support for political parties?”

Fixed-Effects Panel Regression

In [32]:
# Select one party (example: VVD)
vvd = panel[ panel["Partij"] == "VVD" ].copy()

# set panel index
vvd = vvd.set_index(["Code", "Jaar"])

y = vvd["VoteShare"]
X = sm.add_constant(vvd[["Crime_rate"]])

# run fixed effects panel regression
model = PanelOLS(
    y,
    X,
    entity_effects=True,   # municipality fixed effects
    time_effects=True      # year fixed effects
)

results = model.fit(cov_type="clustered", cluster_entity=True)
print(results.summary)


                          PanelOLS Estimation Summary                           
Dep. Variable:              VoteShare   R-squared:                     8.964e-05
Estimator:                   PanelOLS   R-squared (Between):              0.0017
No. Observations:                1916   R-squared (Within):               0.0050
Date:                Thu, Nov 27 2025   R-squared (Overall):              0.0019
Time:                        12:17:05   Log-likelihood                    5020.9
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      0.1301
Entities:                         460   P-value                           0.7184
Avg Obs:                       4.1652   Distribution:                  F(1,1451)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             0.0650
                            

/home/kaai/uva/sda/SDA2025/.venv/lib/python3.12/site-packages/linearmodels/panel/model.py:1258: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


# Subquestion 2
“Do increases and decreases in crime affect voting behaviour symmetrically?”

Interaction Model

In [33]:
# Choose political party for analysis
party_name = "VVD"
df = panel[ panel["Partij"] == party_name ].copy()

# Sort by city and year
df = df.sort_values(["Code", "Jaar"])

# ------------------------------------------------------------
# 1. Compute year-on-year change in crime rate
# ------------------------------------------------------------
df["Crime_diff"] = df.groupby("Code")["Crime_rate"].diff()

# ------------------------------------------------------------
# 2. Dummy indicating whether crime increased (> 0)
# ------------------------------------------------------------
df["Crime_up_dummy"] = (df["Crime_diff"] > 0).astype(int)

# ------------------------------------------------------------
# 3. Interaction term for asymmetry
# ------------------------------------------------------------
df["Crime_diff_x_up"] = df["Crime_diff"] * df["Crime_up_dummy"]

# ------------------------------------------------------------
# 4. Prepare the panel regression
# ------------------------------------------------------------
# from linearmodels.panel import PanelOLS
# import statsmodels.api as sm

df = df.set_index(["Code", "Jaar"])

y = df["VoteShare"]
X = sm.add_constant(df[["Crime_diff", "Crime_diff_x_up"]])

model = PanelOLS(
    y,
    X,
    entity_effects=True,
    time_effects=True
)

results = model.fit(cov_type="clustered", cluster_entity=True)
print(results.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:              VoteShare   R-squared:                        0.0247
Estimator:                   PanelOLS   R-squared (Between):             -0.0015
No. Observations:                1454   R-squared (Within):              -0.0098
Date:                Thu, Nov 27 2025   R-squared (Overall):             -0.0057
Time:                        12:17:08   Log-likelihood                    3883.1
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      12.853
Entities:                         432   P-value                           0.0000
Avg Obs:                       3.3657   Distribution:                  F(2,1017)
Min Obs:                       1.0000                                           
Max Obs:                       4.0000   F-statistic (robust):             9.4096
                            

/home/kaai/uva/sda/SDA2025/.venv/lib/python3.12/site-packages/linearmodels/panel/model.py:1258: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
